# NAMA : RAKHMAT FADHILAH
# NIM : 202210370311442
# KELAS : PEMROSESAN BAHASA ALAMI C
# TUGAS : SCRAPPING DATA TEKS WITH WEBSITE

#### 1. PENDAHULUAN

##### Proyek ini bertujuan untuk mengumpulkan data teks dari berbagai sumber berita online yang membahas topik "judi online" menggunakan teknik web scraping

#### 2. TUJUAN

* Mengumpulkan data teks dari berbagai situs berita yang membahas topik judi online.
* Membersihkan dan memproses teks untuk analisis lebih lanjut.
* Menyimpan data dalam format CSV untuk keperluan penelitian atau analisis.

#### 3. Persyaratan Tugas
1. Memilih sumber data untuk scraping
   - Twitter (melalui API)
   - Situs berita (melalui web scraping)
   - Makalah penelitian (abstrak Google Scholar)

2. Scrape & Pra-pemrosesan Data
   - Mengumpulkan minimal 6000 sampel teks
   - Membersihkan teks (menghapus stopwords, tanda baca, mengubah ke huruf kecil)

#### 4. TOPIK YANG SAYA BAHAS
- DISIN SAYA MENGAMBIL DARI BERBAGAI BERITA MEDIA DI INDONESIA DAN DI BERBAGAI NEGARA YANG MEMBAHAS "Regulasi pada Data Berita Judi Online di Berbagai Negara"

#### Instalasi dan Impor Library
* BeautifulSoup

	BeautifulSoup adalah library Python yang digunakan untuk mengekstrak data dari file HTML dan XML. Dalam proyek ini, BeautifulSoup digunakan untuk mengambil teks dari halaman web dengan mem-parsing struktur HTML.

	Cara Kerja :
	-	BeautifulSoup membaca konten HTML dari halaman web.
	-	Mengidentifikasi tag HTML seperti Elemet (p, h1, h2, article), dan lainnya yang mengandung teks. Mengekstrak teks dari tag-tag tersebut.
	-	Requests
			Requests adalah library Python yang digunakan untuk mengirim permintaan HTTP ke server web. Dalam proyek ini, Requests digunakan untuk mengambil konten HTML dari URL yang ditargetkan.

* Request

	Requests adalah library Python yang digunakan untuk mengirim permintaan HTTP ke server web. Dalam proyek ini, Requests digunakan untuk mengambil konten HTML dari URL yang ditargetkan.

	Cara Kerja : 
	-	Mengirim permintaan GET ke URL.
	-	Menerima respons dari server dalam bentuk HTML.
	-	Mengembalikan konten HTML untuk diproses oleh BeautifulSoup.

* Pandas

	Pandas adalah library Python yang digunakan untuk manipulasi dan analisis data. Dalam proyek ini, Pandas digunakan untuk menyimpan data teks yang telah di-scrape ke dalam format CSV.

	Cara Kerja : 
	-	Data teks disimpan dalam struktur DataFrame.
	-	DataFrame dapat diekspor ke file CSV menggunakan fungsi to_csv().
* NLTK (Natural Language Toolkit)
	
	NLTK adalah library Python yang digunakan untuk pemrosesan bahasa alami (NLP). Dalam proyek ini, NLTK digunakan untuk penghapusan stopwords, yaitu menghilangkan kata-kata umum yang tidak memiliki makna signifikan seperti "dan", "atau", dan "di". Selain itu, NLTK juga digunakan untuk tokenisasi, yaitu memecah teks menjadi kata-kata individual, serta pembersihan teks dengan menghapus karakter khusus, tanda baca, dan format yang tidak diperlukan.
* tqdm dan fake_useragent

	Library tambahan yang digunakan untuk meningkatkan efisiensi teknik scraping

In [1]:
!pip install requests beautifulsoup4 pandas tqdm fake_useragent nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

#### Download NLTK Resource dan Fungsi Pembersihan Teks

Disini download resource nya dulu yaitu stopwords dan tokenizer, Setelah itu Dari beberapa teks atau kata yang diambil akan di proses sesuai dengan kebutuhan tugas yaitu :

1. Mengubah teks menjadi huruf kecil
2. Menghapus tanda baca
3. Menghapus stopwords
4. Melakukan tokenisasi teks

In [3]:
# Download NLTK resources (stopwords dan tokenizer)
nltk.download("punkt")
nltk.download("stopwords")

# Fungsi untuk membersihkan teks
def clean_text(text):
    # Hapus mention (@username)
    text = re.sub(r"@\w+", "", text)
    # Hapus URL
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Hapus tanda baca dan karakter khusus
    text = re.sub(r"[^\w\s]", "", text)
    # Ubah ke lowercase
    text = text.lower()
    # Hapus stopwords
    stop_words = set(stopwords.words("indonesian"))  # Gunakan stopwords bahasa Indonesia
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

[nltk_data] Downloading package punkt to C:\Users\rakhmat
[nltk_data]     fadhilah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\rakhmat
[nltk_data]     fadhilah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Fungsi untuk Scraping Website

Fungsi ini bertugas untuk melakukan scraping dari satu URL dengan menggunakan requests.get untuk mengakses halaman web. Jika permintaan berhasil dengan status kode 200, maka teks diambil dari elemen HTML seperti Element (p, h1, div, dan lainnya). Teks yang telah diambil kemudian dibersihkan menggunakan fungsi clean_text agar lebih rapi dan siap untuk digunakan. Namun, jika proses scraping gagal, fungsi akan mengembalikan list kosong serta mencetak pesan error untuk memberi tahu pengguna mengenai kegagalan tersebut.


Dan disini juga, saya menggunakan 100 Link Website lebih , ada yang dari web berita di indonesia dan ada juga yang di luar negeri

In [4]:
# Fungsi untuk melakukan web scraping dari satu URL
def scrape_website(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=60)  # Timeout 60 detik
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Ambil semua teks dari tag yang relevan (sesuaikan dengan struktur website)
            texts = []
            for element in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6", "article", "div"]):
                if element.text.strip():  # Hanya ambil teks yang tidak kosong
                    cleaned_text = clean_text(element.text.strip())
                    texts.append(cleaned_text)
            
            return texts
        else:
            print(f"Gagal mengakses {url}. Kode status: {response.status_code}")
            return []
    except requests.RequestException as e:
        print(f"Error saat mengakses {url}: {e}")
        return []

# Daftar URL situs web berita di Indonesia yang membahas judi online
urls = [
        "https://www.cnbcindonesia.com/tag/judi-online",
    "https://mediaindonesia.com/tag/judi-online",
    "https://www.kompas.tv/tag/judi-online",
    "https://www.inilah.com/tag/judi-online",
    "https://www.beritasatu.com/tag/judi-online",
    "https://www.sindonews.com/tag/judi-online",
    "https://www.viva.co.id/tag/judi-online",
    "https://www.okezone.com/tag/judi-online",
    "https://www.idntimes.com/tag/judi-online",
    "https://www.medcom.id/tag/judi-online",
    "https://www.tribunnews.com/tag/judi-online",
    "https://www.kumparan.com/tag/judi-online",
    "https://www.merdeka.com/tag/judi-online",
    "https://www.suara.com/tag/judi-online",
    "https://www.antaranews.com/tag/judi-online",
    "https://www.republika.co.id/tag/judi-online",
    "https://www.tempo.co/tag/judi-online",
    "https://www.jpnn.com/tag/judi-online",
    "https://www.balipost.com/tag/judi-online",
    "https://www.klikpositif.com/tag/judi-online",
    "https://www.papuapos.com/tag/judi-online",
    "https://www.jurnaljatim.com/tag/judi-online",
    "https://www.radarlampung.co.id/tag/judi-online",
    "https://www.bengkuluexpress.com/tag/judi-online",
    "https://www.tugumalang.id/tag/judi-online",
    "https://www.kabarbanten.com/tag/judi-online",
    "https://www.pasundanekspres.co/tag/judi-online",
    "https://www.kaltimkece.id/tag/judi-online",
    "https://www.radarmadura.id/tag/judi-online",
    "https://www.lintasbarometer.com/tag/judi-online",
    "https://www.kompas.com/tag/judi-online",
    "https://www.detik.com/tag/judi-online",
    "https://www.liputan6.com/tag/judi-online",
    "https://www.cnnindonesia.com/tag/judi-online",
    "https://www.tribunnews.com/tag/judi-online",
    "https://www.kumparan.com/tag/judi-online",
    "https://www.merdeka.com/tag/judi-online",
    "https://www.suara.com/tag/judi-online",
    "https://www.antaranews.com/tag/judi-online",
    "https://www.republika.co.id/tag/judi-online",
    "https://www.tempo.co/tag/judi-online",
    "https://www.jpnn.com/tag/judi-online",
    "https://www.balipost.com/tag/judi-online",
    "https://www.klikpositif.com/tag/judi-online",
    "https://www.papuapos.com/tag/judi-online",
    "https://www.jurnaljatim.com/tag/judi-online",
    "https://www.radarlampung.co.id/tag/judi-online",
    "https://www.bengkuluexpress.com/tag/judi-online",
    "https://www.tugumalang.id/tag/judi-online",
    "https://www.kabarbanten.com/tag/judi-online",
    "https://www.pasundanekspres.co/tag/judi-online",
    "https://www.kaltimkece.id/tag/judi-online",
    "https://www.radarmadura.id/tag/judi-online",
    "https://www.lintasbarometer.com/tag/judi-online",
    "https://www.riausky.com/tag/judi-online",
    "https://www.rmol.id/tag/judi-online",
    "https://www.inilah.com/tag/judi-online",
    "https://www.beritasatu.com/tag/judi-online",
    "https://www.ayobandung.com/tag/judi-online",
    "https://www.sindonews.com/tag/judi-online",
    "https://www.viva.co.id/tag/judi-online",
    "https://www.radarcirebon.com/tag/judi-online",
    "https://www.batamnews.co.id/tag/judi-online",
    "https://www.kaltimpost.co.id/tag/judi-online",
    "https://www.borneonews.co.id/tag/judi-online",
    "https://www.fajar.co.id/tag/judi-online",
    "https://www.sulselprov.go.id/tag/judi-online",
    "https://www.jawapos.com/tag/judi-online",
    "https://www.ekonomibisnis.com/tag/judi-online",
    "https://www.harianjogja.com/tag/judi-online",
    "https://www.solopos.com/tag/judi-online",
    "https://www.rri.co.id/tag/judi-online",
    "https://www.gatra.com/tag/judi-online",
    "https://www.idntimes.com/tag/judi-online",
    "https://www.katakini.com/tag/judi-online",
    "https://www.medcom.id/tag/judi-online",
    "https://www.acehtrend.com/tag/judi-online",
    "https://www.aktual.com/tag/judi-online",
    "https://www.teropongsenayan.com/tag/judi-online",
    "https://www.pikiran-rakyat.com/tag/judi-online",
    "https://www.krjogja.com/tag/judi-online",
    "https://www.timesindonesia.co.id/tag/judi-online",
    "https://www.republika.co.id/indeks/hukum/kriminalitas/judi-online",
    "https://news.detik.com/berita/d-5697180/situs-situs-pemerintah-disusupi-judi-online",
    "https://www.kompas.id/baca/investigasi/2023/12/05/kisah-mereka-dari-balik-situs-judi",
    "https://nasional.kompas.com/read/2024/06/21/15541661/polri-bongkar-judi-online-di-3-situs-18-tersangka-ditangkap",
    "https://www.hukumonline.com/klinik/a/ikut-judi-online-dari-luar-negeri-bisa-dipidana-lt5fabd3a5f21a8/",
    "https://www.rri.co.id/daerah/755425/media-sosial-ladang-penyebaran-konten-judi-online",
    "https://www.detik.com/jabar/berita/d-7540664/waspadai-ini-5-modus-penipuan-judi-online",
    "https://rri.co.id/nasional/657869/server-judi-online-di-luar-negeri-satgas-gandeng-interpol",
    "https://dpd.go.id/daftar-berita/senator-ntt-judi-online-menyengsarakan-rakyat",
    "https://www.detik.com/tag/judi-online-internasional",
    "https://www.cnnindonesia.com/nasional/2024/08/19/080000/polisi-bongkar-jaringan-judi-online-omzet-rp-2-miliar",
    "https://www.kompas.com/hype/read/2024/09/15/200000/penyanyi-terkenal-terjerat-kasus-judi-online",
    "https://www.liputan6.com/news/read/4630000/polda-metro-jaya-tangkap-bandar-judi-online-beromzet-miliaran",
    "https://www.suara.com/news/2024/07/22/180000/mahasiswa-tertangkap-ikut-judi-online-polisi-beri-imbauan",
    "https://www.merdeka.com/peristiwa/kejaksaan-agung-tangkap-buron-kasus-judi-online-di-singapura.html",
    "https://www.antaranews.com/berita/2320000/kemkominfo-blokir-1000-situs-judi-online",
    "https://www.tribunnews.com/nasional/2024/10/10/kapolri-tegaskan-akan-tindak-tegas-pelaku-judi-online",
    "https://www.kumparan.com/berita/polisi-ungkap-modus-baru-judi-online-melalui-aplikasi-perpesanan",
    "https://www.viva.co.id/berita/nasional/1400000-kasus-judi-online-meningkat-selama-pandemi-covid-19",
    "https://www.sindonews.com/read/500000/13/polisi-bongkar-jaringan-judi-online-internasional",
    "https://www.okezone.com/tag/judi-online",
    "https://www.beritasatu.com/nasional/800000/polisi-tangkap-20-tersangka-judi-online-di-jakarta",
    "https://www.idntimes.com/news/indonesia/polisi-bongkar-kasus-judi-online-di-sumatera-utara",
    "https://www.medcom.id/nasional/hukum/8N0a0000-polisi-tangkap-bandar-judi-online-di-bali",
    "https://www.tempo.co/nasional/2024/09/01/polisi-gerebek-markas-judi-online-di-surabaya",
    "https://www.krjogja.com/berita-lokal/read/500000/polisi-ungkap-kasus-judi-online-di-yogyakarta",
    "https://www.solopos.com/polisi-tangkap-agen-judi-online-di-solo-1130000",
    "https://www.ayobandung.com/read/2024/08/15/500000/polisi-bongkar-jaringan-judi-online-di-bandung",
    "https://www.bbc.com/news/topics/c40rjmqdl5jt/gambling",
    "https://www.theguardian.com/society/gambling",
    "https://www.nytimes.com/topic/subject/gambling",
    "https://www.cnbc.com/gambling/",
    "https://www.reuters.com/business/gambling/",
    "https://www.forbes.com/sites/gambling/",
    "https://www.abc.net.au/news/topic/gambling",
    "https://www.independent.co.uk/topic/gambling",
    "https://www.aljazeera.com/tag/gambling/",
    "https://www.foxnews.com/category/lifestyle/gambling",
    "https://calvinayre.com/",
    "https://www.yogonet.com/international",
    "https://www.gamblingnews.com/",
    "https://www.igamingbusiness.com/",
    "https://www.vegasslotsonline.com/news/",
    "https://www.gamblinginsider.com/news",
    "https://www.casino.org/news/",
    "https://www.pokernews.com/news/",
    "https://www.gamingtoday.com/news/",
    "https://www.gambling.com/news",
    "https://www.onlinegambling.com/news/",
    "https://www.casinonewsdaily.com/",
    "https://www.gamblinginsider.com/news",
    "https://www.gamingintelligence.com/",
    "https://www.igamingbusiness.com/news",
    "https://www.pokernews.com/news/",
    "https://www.gamblingnews.com/",
    "https://www.casino.org/news/",
    "https://www.americangaming.org/",
    "https://www.casinobeats.com/",
    "https://www.egr.global/",
    "https://www.lasvegasadvisor.com/news/",
    "https://www.legaluspokersites.com/news/",
    "https://www.sportsbookreview.com/news/",
    "https://www.worldcasinodirectory.com/news/",
    "https://www.igamingbrazil.com/",
    "https://www.top10casinos.com/news/",
    "https://www.bestuscasinos.org/news/",
    "https://www.usgamblingsites.com/news/",
    "https://www.gambling911.com/",
    "https://www.askgamblers.com/gambling-news",
    "https://www.casinoguardian.co.uk/",
    "https://www.ukgamblingsites.com/news/",
    "https://www.nongamstopcasinos.net/news/",
    "https://www.casinosites.org/news/",
    "https://www.usaonlinecasino.com/news/",
    "https://www.sportsbettingdime.com/news/",
    "https://www.vegasinsider.com/news/",
    "https://www.casinolistings.com/news/",
    "https://www.casinovalley.ca/news/",
    "https://www.onlinepokerreport.com/",
    "https://www.playpennsylvania.com/news/",
    "https://www.betpicks.ca/news/"
        "https://www.bbc.com/news/topics/c40rjmqdl5jt/gambling",  # BBC News - Gambling
    "https://www.theguardian.com/society/gambling",  # The Guardian - Gambling
    "https://www.nytimes.com/topic/subject/gambling",  # The New York Times - Gambling
    "https://www.cnbc.com/gambling/",  # CNBC - Gambling News
    "https://www.reuters.com/business/gambling/",  # Reuters - Gambling Industry News
    "https://www.forbes.com/sites/gambling/",  # Forbes - Gambling Industry
    "https://www.abc.net.au/news/topic/gambling",  # ABC Australia - Gambling
    "https://www.independent.co.uk/topic/gambling",  # The Independent - Gambling
    "https://www.aljazeera.com/tag/gambling/",  # Al Jazeera - Gambling
    "https://www.foxnews.com/category/lifestyle/gambling",  # Fox News - Gambling
    "https://calvinayre.com/",  # CalvinAyre - Gambling Industry News
    "https://www.yogonet.com/international",  # Yogonet - International Gambling News
    "https://www.gamblingnews.com/",  # GamblingNews - Industry Updates
    "https://www.igamingbusiness.com/",  # iGaming Business - Gambling Industry Analysis
    "https://www.vegasslotsonline.com/news/",  # Vegas Slots Online - Gambling & Casino News
    "https://www.gamblinginsider.com/news",  # Gambling Insider - Industry Updates
    "https://www.casino.org/news/",  # Casino.org - Gambling News
    "https://www.pokernews.com/news/",  # PokerNews - Gambling & Poker Industry
    "https://www.gamingtoday.com/news/",  # Gaming Today - Gambling News
    "https://www.gambling.com/news",  # Gambling.com - Berita dan panduan perjudian
    "https://www.onlinegambling.com/news/",  # OnlineGambling.com - Berita perjudian online
    "https://www.casinonewsdaily.com/",  # Casino News Daily - Berita kasino dan perjudian
    "https://www.gamblinginsider.com/news",  # Gambling Insider - Berita industri perjudian
    "https://www.gamingintelligence.com/",  # Gaming Intelligence - Berita dan analisis industri perjudian
    "https://www.igamingbusiness.com/news",  # iGaming Business - Berita industri perjudian online
    "https://calvinayre.com/",  # CalvinAyre - Berita industri perjudian
    "https://www.pokernews.com/news/",  # PokerNews - Berita poker dan perjudian
    "https://www.gamblingnews.com/",  # GamblingNews - Berita industri perjudian
    "https://www.casino.org/news/",  # Casino.org - Berita dan panduan kasino
    "https://www.pokernews.com/news/",
    "https://www.gamblingnews.com/",
    "https://www.igamingbusiness.com/",
    "https://www.vegasslotsonline.com/news/",
    "https://www.gamblinginsider.com/news",
    "https://www.casino.org/news/",
    "https://www.gamingtoday.com/news/",
    "https://www.gambling.com/news",
    "https://www.onlinegambling.com/news/",
    "https://www.casinonewsdaily.com/",
    "https://www.gamingintelligence.com/",
    "https://www.casinobeats.com/",
    "https://www.egr.global/",
    "https://www.lasvegasadvisor.com/news/",
    "https://www.legaluspokersites.com/news/",
    "https://www.sportsbookreview.com/news/",
    "https://www.worldcasinodirectory.com/news/",
    "https://www.igamingbrazil.com/",
    "https://www.top10casinos.com/news/",
    "https://www.bestuscasinos.org/news/",
    "https://www.usgamblingsites.com/news/",
    "https://www.gambling911.com/",
    "https://www.askgamblers.com/gambling-news",
    "https://www.casinoguardian.co.uk/",
    "https://www.ukgamblingsites.com/news/",
    "https://www.nongamstopcasinos.net/news/",
    "https://www.casinosites.org/news/",
    "https://www.usaonlinecasino.com/news/",
    "https://www.sportsbettingdime.com/news/",
    "https://www.vegasinsider.com/news/",
    
]


## Penyimpanan Data ke dalam excel berbentuk csv

Sebelum di simpan ke dalam file CSV ada beberapa yang saya saring terlebih dahulu, agar data teks yang ada di dalam CSV nanti itu tidak acak-acakan, berikut tahapannya:

- Hapus Url yang sama

    Disini saya hapus Url yang sama maksud nya, dikarenakan saya meminta kepada 2 AI , ChatGPT dan Deepseek untuk mencarikan website yang dimana membahas sebuah Topik judi online, nah dari kedua AI tersebut alangkah bisa terjadi url yang sama, jadi jika ada url yang sama, maka akan di skip otomatis

- Hapus teks yang duplikat

    Ketika saya tidak menghapus teks yang duplikat, maka nanti bisa saja data yang masuk di excel nya itu adalah data yang duplikat, jadi akan membuat struktur data teks menjadi jelek.

- 4 Kata(Opsional)

    Jadi disini saya mengahruskan panjang kata yang dimasukan ke dalam File CSV saya itu minimal lebih dari 4 kata, karena ketika saya tidak menggunakan ini, ada 1 kata saja yang masuk ke dalam baris di file CSV saya.

Setelah semua proses sudah dilaksanakan, baru akan disimpan ke dalam sebuah file CSV dengan nama file "Scraping_Article_Web_about_Judi_Online.csv
Scraping_Article_Web_about_Judi_Online.csv"

Data yang saya dapatkan sebelum saya hapus duplikatnya dan lebih dari 4 kata yang masuk ke dalam CSV dari seluruh website adalah 9000 lebih dan data yang sudah di proses sama saya menjadi 6000 lebih.



In [ ]:
# Hapus duplikat URL
unique_urls = list(set(urls))

# Kumpulkan teks unik
all_texts = set()  # Gunakan set() agar otomatis menghapus duplikat
for url in unique_urls:
    print(f"Scraping dari: {url}")
    texts = scrape_website(url)
    all_texts.update(texts)  # Menambahkan ke set agar tidak ada duplikat
    print(f"Berhasil mengumpulkan {len(texts)} sampel teks dari {url}")
    print(f"Total teks unik sejauh ini: {len(all_texts)}")

    if len(all_texts) >= 10000:
        print("Sudah mencapai 6000 data. Proses scraping dihentikan.")
        break

# Konversi ke list dan simpan hanya 6000 data unik
final_texts = [text for text in all_texts if len(text.split()) > 4][:7000]
df = pd.DataFrame(final_texts, columns=["text"])
df.to_csv("Scraping_Article_Web_about_Judi_Online.csv", index=False, encoding="utf-8")

print(f"Total data unik yang disimpan: {len(final_texts)}")
print("Data telah disimpan ke Scraping_Article_Web_about_Judi_Online.csv")


Scraping dari: https://www.aktual.com/tag/judi-online
Berhasil mengumpulkan 339 sampel teks dari https://www.aktual.com/tag/judi-online
Total teks unik sejauh ini: 132
Scraping dari: https://www.abc.net.au/news/topic/gambling
Berhasil mengumpulkan 315 sampel teks dari https://www.abc.net.au/news/topic/gambling
Total teks unik sejauh ini: 325
Scraping dari: https://www.americangaming.org/
Gagal mengakses https://www.americangaming.org/. Kode status: 403
Berhasil mengumpulkan 0 sampel teks dari https://www.americangaming.org/
Total teks unik sejauh ini: 325
Scraping dari: https://www.teropongsenayan.com/tag/judi-online
Error saat mengakses https://www.teropongsenayan.com/tag/judi-online: HTTPSConnectionPool(host='www.teropongsenayan.com', port=443): Max retries exceeded with url: /tag/judi-online (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Berhasil mengumpulkan 0 samp